# Demo: Control Flow and Conditional Execution Concepts
In this demonstration, we will explore different control flow an conditional execution concepts, such as if-else statements, and short-circuit evaluation. Let's start with some definitions:
* The __if-else paradigm__ is a control flow construct where a boolean condition is evaluated, and if it is `true`, a specific block of code is executed; otherwise, if it is `false` an alternative block (the else) may run. It enables branching logic by directing program execution along different paths based on boolean conditions.
* __Short-circuit evaluation__ is a paradigm in which multiple boolean cases must be checked, where chains of logical expressions (separed by AND or OR opeators e.g., `&&` and `||`) are evaluated left-to-right and stop as soon as the overall result is determined, preventing evaluation of unnecessary operands. This improves efficiency and avoids unintended side effects by not executing expressions that won’t affect the final outcome.

While we are going to show some examples in Julia, these concepts are common in many (if not all) programming languages.
* __History__: The if-else paradigm originated from early branch instructions and arithmetic IF constructs in first-generation languages like Fortran in the 1950s, and was formalized into structured if-then-else blocks in ALGOL around 1960, promoting clearer control flow over goto-based logic. The idea of conditional expressions was further advanced by John McCarthy in his late-1950s work on Lisp, embedding if-then-else as a fundamental construct in symbolic processing and influencing modern programming languages’ control.

To demonstrate these concepts, we'll build a simple student record application that stores student information, and some logic to search for particular students are collections of students. Let's go!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.
* The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 
* In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl), check out that documentation for more information on the functions and types used in this material.

In [1]:
include("Include.jl");

### Types and Functions
Let's start by defining mutable `MySimpleStudentModel` type, which will represent a student record. This type will include fields for the student's first name, last name, student identification number, and thier netid (email). 
* _Why a mutable type?_ Mutable types allow for modification of their fields after creation, which is useful for objects that may need to be updated or changed over time, such as student records. Additonally, mutable types offer different possible initialization options, such as default values for fields.
* _Keyword argument constructor_: The constructor allows for the creation of `MySimpleStudentModel` objects with default values for fields, making it easier to instantiate objects without providing all arguments. Thus, we can create an empty student record with default values for each field.

In [2]:
"""
    mutable struct MySimpleStudentModel

A mutable struct that models a student with a firstname, lastname, student id and a netid.

### Fields
- `firstname::String`: The first name of the student.
- `lastname::String`: The last name of the student.
- `sid::Int64`: The student identification number.
- `netid::String`: The network identifier (email address) of the student.
"""
mutable struct MySimpleStudentModel

    # data fields -
    firstname::String
    lastname::String
    sid::Int64
    netid::String
    
    # keyword argument constructor: builds a new student model with default values
    MySimpleStudentModel(; firstname::String = "firstname", 
        lastname::String = "lastname", sid::Int64 = -1, netid::String = "abcd") = new(firstname, lastname, sid, netid);
end;

We can build an empty (each field is set to a default value) student record as follows by calling the constructor with no arguments. Let's do thay and save the default student model in a variable `teststudent`:
```julia

In [9]:
teststudent = MySimpleStudentModel(); # all the fields are set to default values

We access the fields of the student record using dot notation, e.g., `student.firstname` to access the `first_name` field of the `student` object:

In [10]:
teststudent.firstname

"firstname"

## Implement a factory method to build and configure a student model instance
In this task, we'll implement a `build(...)` method that builds and configures a `MySimpleStudentModel` struct. 
* __Pattern__: In the rest of the course, we will use the `build(...)` method pattern to create and configure instances of various composite types. This pattern is useful for encapsulating the logic of creating and initializing objects, especially when there are multiple ways to construct an object or when additional configuration, or validation is needed.

Let's implement a `build(model::Type{MySimpleStudentModel}; data::NamedTuple)::MyStudentModel` method which takes the type of thing we want to build, and the data need to build the model [in a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple). The `build(...)` returns a populated `MySimpleStudentModel` instance.

In [17]:
function build(modeltype::Type{MySimpleStudentModel}, data::NamedTuple)::MySimpleStudentModel
    
    # initailize -
    model = modeltype(); # This builds an empty model (with default values)

    # TODO: Unomment the following lines to set the fields of the model
    # get data from the named tuple and set the fields of the model
    model.firstname = data.firstname;
    model.lastname = data.lastname;
    model.sid = data.sid;
    model.netid = data.netid;
    
    # return -
    return model;
end;

### TODO: Build a test student model
Once we implemented our `build(...)` method, construct a `MySimpleStudentModel` instance with specified values for the fields of the model. Let's save the result in the `test_student_model::MySimpleStudentModel` variable, and the data we used to build the model in a `NamedTuple` instance called `test_student_data`.

```julia

In [27]:
test_student_model, test_student_data = let

    # Specify the data for the model
    data = (firstname = "John", lastname = "Doe", sid = 123456, netid = "jd235");
    model = build(MySimpleStudentModel, data);

    model, data; # return the model
end;

In [26]:
test_student_model

MySimpleStudentModel("John", "Doe", 123456, "jd235")

### TODO: Test your student model instance
Now that we have constructed our `test_student_model` instance let's develop [some unit tests](https://en.wikipedia.org/wiki/Unit_testing) to ensure the model's type and data are correct.
* __What are [unit tests](https://en.wikipedia.org/wiki/Unit_testing)__? Unit testing is a form of software testing by which isolated source code components, a.k.a units, are tested to validate expected behavior. This form of testing is a ground-up perspective in which we validate each system unit.

The first test we do is to determine whether the correct build method is being called by validating the type of model object being returned using [the `isa(...)` method](https://docs.julialang.org/en/v1/base/base/#Core.isa). 

In [21]:
@assert isa(test_student_model, MySimpleStudentModel) # check if the model is of type MySimpleStudentModel

Next, test whether the values fields of the student model are the correct types and have the correct values. These tests use the short-circuit `&&` operator to verify the value of each field. 
* _What_? The logical `&&` operator takes two boolean expressions and evaluates them left-to-right, returning `true` if both are `true`, otherwise it returns `false`. If the first expression is `false`, the second expression is not evaluated (short-circuiting). For more information on `&&` check-out the [Julia Short-Circuit conditional statement documentation](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation). 

First, we'll test the types of the data fields using [the `isa(...)` method](https://docs.julialang.org/en/v1/base/base/#Core.isa) in combination with [the logical and operator `&&`](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation):

In [22]:
@assert (isa(test_student_model.sid, Int) && isa(test_student_model.netid, String) && 
         isa(test_student_model.firstname, String) && isa(test_student_model.lastname, String)) # check if the fields are of the correct typ

Then test the values held in the data fields using the equality `==` operator:

In [28]:
@assert test_student_model.firstname == test_student_data.firstname
@assert test_student_model.lastname == test_student_data.lastname
@assert test_student_model.sid == test_student_data.sid
@assert test_student_model.netid == test_student_data.netid

or using [the logical and operator `&&`](https://docs.julialang.org/en/v1/manual/control-flow/#Short-Circuit-Evaluation). What should these values be?

In [29]:
@assert (test_student_model.firstname == test_student_data.firstname && 
        test_student_model.lastname == test_student_data.lastname &&
        test_student_model.sid == test_student_data.sid &&
        test_student_model.netid == test_student_data.netid);

## Debug a student record search function
In this task, we will debug an existing implementation of the `find(...)` method which returns the index of a student object by matching the `sid::Int64` and `netid::String` fields with a test student object.

First, let's finish the implementation of the [`find(...)` method](src/Compute.jl). This method takes a collection of student models and values for the `sid::Int` and `netid::String` fields and returns either the index of the student object in the collection or `nothing`.

In [ ]:
"""
Fill me in.
"""
function find(students::Array{MySimpleStudentModel,1}, 
    sid::Int64 = 0, netid::String="jdv27")::Union{Int64, Nothing}

    # check: should we check to see if the parameters are valid?
    # Let's discuss this in the testing phase ...

    # initialize -
    student_index = nothing; # default: we don't know which student we are looking for

    # main loop -
    for i ∈ 1:eachindex(students)
        test_student = students[i];  # get student i from the array -

        # if statement: does test_student have the same sid and netid as the student that was passed in? 
        # what is the && operator doing here?
        if test_student.sid == sid && test_student.netid == netid
            student_index = i; # we found the student, so set the index to i
            break; # exit the loop
        end
    end

    return student_index; # return the search results to the caller
end;

Next, let's generate some test data. We'll build an array of `MySimpleStudentModel` instances, with random values for the fields. We'll then use random examples from this array and see if the `find(...)` method returns the same index as the random index.

In [ ]:
list_of_test_students = let
    
    # initialize -
    number_of_test_students = 1000; 
    list_of_test_students = Array{MySimpleStudentModel,1}(undef, number_of_test_students);

end